<a href="https://colab.research.google.com/github/BartoszJanJerzy/stats/blob/main/stata_AW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WELCOME!

I (Bartosz Wójtowicz, The John Paul II Catholic University of Lublin) wrote this this code to calculate results in one of master thesis.

---

What is the scehdule?

1 - import libraries

2 - functions used later

3 - import data and screening

4 - reliability coeffs (Cronbach's alpha)

5 - select subscale to analyze

6 - correlation matrix

7 - main analysis: linear regressions

8 - extra analysis: curvilinear regressions

---

We share this code with you, feel free to use or check it.

#1 - LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr, ttest_rel
import statistics
import statsmodels.api as sm # import statsmodels
from scipy import stats

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



#2 - FUNCTIONS

#3 - DATA

##Load data

In [33]:
raw_data = pd.read_excel('data.xlsx','Data')
data = raw_data.copy()
data.drop('Sygnatura czasowa', axis=1, inplace=True)
data.head()

,wiek,miejsce_zamieszkania,wyksztalcenie,stan_cywilny,aktywnosc_zawodowa,wyk_medyczne,pyt_1.1,pyt_2.1,pyt_3.1,pyt_4.1,pyt_5.1,pyt_6.1,pyt_7.1,pyt_8.1,pyt_9.1,pyt_10.1,pyt_11.1,pyt_12.1_kichanie,pyt_12.1_kaszel,pyt_12.1_schody,pyt_12.1_spacer,pyt_12.1_schody.1,pyt_13.1,pyt_14.1_oslabienie,pyt_14.1_porody,pyt_14.1_niskistatus,pyt_14.1_infekcje,pyt_14.1_1,pyt_14.1_papierosy,pyt_15.1,pyt_16.1,pyt_17.1,pyt_18.1,pyt_19.1,pyt_20.1,pyt_21.1,pyt_22.1,pyt_23.1_higiena,pyt_23.2_leczeniewad,pyt_23.3_masaciala,pyt_23.4_unikaniezaparc,pyt_23.5_cwiczenie,pyt_24.1,pyt_25.1,pyt_26.1,izz_1,izz_2,izz_3,izz_4,izz_5,izz_6,izz_7,izz_8,izz_9,izz_10,izz_11,izz_12,izz_13,izz_14,izz_15,izz_16,izz_17,izz_18,izz_19,izz_20,izz_21,izz_22,izz_23,izz_24,izz_25
0,2,2,2,1,1,1,1,4,1,1,NaN,NaN,1,1,1,1,1,1,1,1,1,1,3,1,1,0,1,0,0,1,1,1,2,2,1,1,1,0,1,0,0,1,1,1,3.0,4,4,4,4,3,5,3,3,3,4,3,3,2,3,4,4,4,3,3,5,3,4,4,3,NaN
1,2,1,2,2,1,0,3,1,1,2,1.0,"Uważam, że problem ten nie wymaga konsultacji ...",3,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,1,2,0,1,0,0,1,1,1,2.0,4,5,3,2,3,4,3,3,3,4,3,1,2,2,4,3,4,2,3,5,4,4,5,2,NaN
2,2,1,2,2,1,0,3,1,4,1,1.0,"Uważam, że problem ten nie wymaga konsultacji ...",4,1,1,1,1,1,1,1,0,0,1,1,1,0,1,0,0,1,1,1,2,2,1,1,2,1,1,1,1,1,1,1,1.0,5,3,5,2,2,3,1,2,4,4,2,2,3,4,5,4,4,5,2,5,5,5,2,3,NaN
3,2,2,2,2,1,1,1,4,3,1,NaN,NaN,1,1,1,2,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,3,2,2,1,1,1,0,0,1,1,1,2,1,2.0,3,4,5,4,2,4,3,3,4,4,4,3,4,2,4,4,4,4,3,5,4,5,4,4,NaN
4,2,2,2,2,1,1,2,1,3,2,1.0,Odkładanie swoich problemów na później,1,1,1,1,1,1,0,1,1,0,3,1,1,0,0,1,0,1,1,1,1,2,1,1,2,1,1,0,0,1,2,1,3.0,4,4,3,3,3,5,3,2,3,3,3,3,4,4,4,2,4,3,3,4,4,5,4,4,NaN


##Make subscales

In [34]:
data['izz_nawyki'] = data['izz_1'] + data['izz_5'] + data['izz_9'] + data['izz_13'] + data['izz_17'] + data['izz_21']
data['izz_profi'] = data['izz_2'] + data['izz_6'] + data['izz_10'] + data['izz_14'] + data['izz_18'] + data['izz_22']
data['izz_psych'] = data['izz_3'] + data['izz_7'] + data['izz_11'] + data['izz_15'] + data['izz_19'] + data['izz_23']
data['izz_zdrow'] = data['izz_4'] + data['izz_8'] + data['izz_12'] + data['izz_16'] + data['izz_20'] + data['izz_24']
data[['izz_nawyki','izz_profi','izz_psych','izz_zdrow']]

,izz_nawyki,izz_profi,izz_psych,izz_zdrow
0,19,23,21,22
1,20,21,21,16
2,23,24,17,18
3,21,23,23,23
4,22,24,20,18
...,...,...,...,...
110,23,25,20,20
111,18,20,21,22
112,20,20,22,21
113,18,22,21,20


In [35]:
print(f'Now - before screening - the sample has N={len(data)} records')
print(f'W tym momencie - przed czyszczeniem danych - próba badawcza posiada N={len(data)} rekordów')

Now - before screening - the sample has N=115 records
W tym momencie - przed czyszczeniem danych - próba badawcza posiada N=115 rekordów


##Screenning

###Calculate means and stds

In [36]:
mean_nawyki = data['izz_nawyki'].mean()
sd_nawyki = data['izz_nawyki'].std()

mean_profi = data['izz_profi'].mean()
sd_profi = data['izz_profi'].std()

mean_psych = data['izz_psych'].mean()
sd_psych = data['izz_psych'].std()

mean_zdrow = data['izz_zdrow'].mean()
sd_zdrow = data['izz_zdrow'].std()

###Make filters

In [37]:
filter_1 = data['izz_nawyki'] > mean_nawyki - 3*sd_nawyki
filter_2 = data['izz_nawyki'] < mean_nawyki + 3*sd_nawyki
filter_3 = data['izz_profi'] > mean_profi - 3*sd_profi
filter_4 = data['izz_profi'] < mean_profi + 3*sd_profi
filter_5 = data['izz_psych'] > mean_psych - 3*sd_psych
filter_6 = data['izz_psych'] < mean_psych + 3*sd_psych
filter_7 = data['izz_zdrow'] > mean_zdrow - 3*sd_zdrow
filter_8 = data['izz_zdrow'] < mean_zdrow + 3*sd_zdrow

###Use filters

In [38]:
data = data[filter_1]
data = data[filter_2]
data = data[filter_3]
data = data[filter_4]
data = data[filter_5]
data = data[filter_6]
data = data[filter_7]
data = data[filter_8]

print(f'Now - after screening - the sample has N={len(data)} records')
print(f'W tym momencie - po czyszczeniem danych - próba badawcza posiada N={len(data)} rekordów')

Now - after screening - the sample has N=115 records
W tym momencie - po czyszczeniem danych - próba badawcza posiada N=115 rekordów


#4 - RESEARCH SAMPLE

##Converse numbers into labels

In [ ]:
df_demo = data[['wiek','miejsce_zamieszkania','wyksztalcenie','stan_cywilny',	'aktywnosc_zawodowa', 'wyk_medyczne']]

for i in range(len(df_demo)):
  # wiek
  if df_demo['wiek'][i] == 1:
    df_demo['wiek'][i] = '20-25 lat'
  elif df_demo['wiek'][i] == 2:
    df_demo['wiek'][i] = '26-35 lat'
  elif df_demo['wiek'][i] == 3:
    df_demo['wiek'][i] = '36-50 lat'
  elif df_demo['wiek'][i] == 4:
    df_demo['wiek'][i] = 'pow. 50 lat'

  # miejsce zamieszkania
  if df_demo['miejsce_zamieszkania'][i] == 1:
    df_demo['miejsce_zamieszkania'][i] = 'Wieś'
  elif df_demo['miejsce_zamieszkania'][i] == 2:
    df_demo['miejsce_zamieszkania'][i] = 'Miasto'
  
  # wykształcenie
  if df_demo['wyksztalcenie'][i] == 1:
    df_demo['wyksztalcenie'][i] = 'Średnie'
  elif df_demo['wyksztalcenie'][i] == 2:
    df_demo['wyksztalcenie'][i] = 'Wyższe'
  
  # stan cywilny
  if df_demo['stan_cywilny'][i] == 1:
    df_demo['stan_cywilny'][i] = 'Panna'
  elif df_demo['stan_cywilny'][i] == 2:
    df_demo['stan_cywilny'][i] = 'Mężatka'
  elif df_demo['stan_cywilny'][i] == 3:
    df_demo['stan_cywilny'][i] = 'Wdowa'
  elif df_demo['stan_cywilny'][i] == 4:
    df_demo['stan_cywilny'][i] = 'Rozwiedziona'
  
  # aktywność zawodowa
  if df_demo['aktywnosc_zawodowa'][i] == 1:
    df_demo['aktywnosc_zawodowa'][i] = 'Aktywna zawodowo'
  elif df_demo['aktywnosc_zawodowa'][i] == 2:
    df_demo['aktywnosc_zawodowa'][i] = 'Aktualnie nie pracuję'
  elif df_demo['aktywnosc_zawodowa'][i] == 3:
    df_demo['aktywnosc_zawodowa'][i] = 'Rencistka/emerytka'
  
  # wykształcenie medyczne
  if df_demo['wyk_medyczne'][i] == 1:
    df_demo['wyk_medyczne'][i] = 'Tak'
  elif df_demo['wyk_medyczne'][i] == 0:
    df_demo['wyk_medyczne'][i] = 'Nie'

In [72]:
df_demo

,wiek,miejsce_zamieszkania,wyksztalcenie,stan_cywilny,aktywnosc_zawodowa,wyk_medyczne
0,26-35 lat,Miasto,Wyższe,Panna,Aktywna zawodowo,Tak
1,26-35 lat,Wieś,Wyższe,Mężatka,Aktywna zawodowo,Nie
2,26-35 lat,Wieś,Wyższe,Mężatka,Aktywna zawodowo,Nie
3,26-35 lat,Miasto,Wyższe,Mężatka,Aktywna zawodowo,Tak
4,26-35 lat,Miasto,Wyższe,Mężatka,Aktywna zawodowo,Tak
...,...,...,...,...,...,...
110,36-50 lat,Miasto,Wyższe,Panna,Aktywna zawodowo,Nie
111,36-50 lat,Miasto,Wyższe,Mężatka,Aktywna zawodowo,Nie
112,20-25 lat,Wieś,Wyższe,Mężatka,Aktywna zawodowo,Nie
113,36-50 lat,Wieś,Wyższe,Mężatka,Aktywna zawodowo,Tak


##Show demographics

In [106]:
categories = ['20-25 lat','26-35 lat','36-50 lat','pow. 50 lat','Wieś','Miasto','Średnie','Wyższe','Panna','Mężatka','Wdowa','Rozwiedziona','Aktywna zawodowo','Aktualnie nie pracuję','Rencistka/emerytka','Tak','Nie']
values_1 = df_demo['wiek'].value_counts(sort=False)
values_2 = df_demo['miejsce_zamieszkania'].value_counts(sort=False).to_list()
values_3 = df_demo['wyksztalcenie'].value_counts(sort=False).to_list()
values_4 = df_demo['stan_cywilny'].value_counts(sort=False).to_list()
values_5 = df_demo['aktywnosc_zawodowa'].value_counts(sort=False).to_list()
values_6 = df_demo['wyk_medyczne'].value_counts(sort=False).to_list()

values = []
percentages = []

for results in [values_1,values_2,values_3,values_4,values_5,values_6]:
  for i in results:
    values.append(i)
    percentages.append(str(round((100*i)/115,1)) + '%')

dict_sample_demo = {
    'cat':categories,
    'N':values,
    '%':percentages
}

sample_demo_desc = pd.DataFrame(dict_sample_demo)
sample_demo_desc

,cat,N,%
0,20-25 lat,43,37.4%
1,26-35 lat,7,6.1%
2,36-50 lat,14,12.2%
3,pow. 50 lat,51,44.3%
4,Wieś,50,43.5%
5,Miasto,65,56.5%
6,Średnie,11,9.6%
7,Wyższe,104,90.4%
8,Panna,2,1.7%
9,Mężatka,80,69.6%


In [107]:
sample_demo_desc.to_excel('demo_stats.xlsx')

#5 - DESCRIPTIVE STATS

In [49]:
desc_stats = data[['izz_nawyki','izz_profi','izz_psych','izz_zdrow']].describe()
desc_stats = desc_stats.drop(['25%','50%','75%','count'])
desc_stats = desc_stats.T
desc_stats

,mean,std,min,max
izz_nawyki,20.626087,3.784690,11.0,30.0
izz_profi,21.782609,3.986342,12.0,30.0
izz_psych,20.704348,4.047902,11.0,29.0
izz_zdrow,19.747826,4.092877,9.0,28.0


In [109]:
desc_stats.to_excel('desc_stats.xlsx')